# Demo SingleRun
This notebook shows a single run consisting of the following four phases:
* sail empty
* loading
* sail full
* unloading

In [1]:
import datetime, time
import simpy

import shapely.geometry


import pandas as pd
import openclsim.core as core
import openclsim.model as model

import openclsim.plugins as plugin

# setup environment
simulation_start  = datetime.datetime(2010,1,1)
my_env = simpy.Environment(initial_time=simulation_start.timestamp())
registry = {}
keep_resources = {}

import numpy as np

In [2]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
    ),
    {},
)

TestMoveActivity =  type(
    "TestMoveActivity",
    (
        plugin.HasWeatherPluginMoveActivity,
        model.MoveActivity,  # the order is critical!
    ),
    {},
)


TestShiftActivity =  type(
    "TestShiftActivity",
    (
        plugin.HasWeatherPluginShiftAmountActivity,
        model.ShiftAmountActivity,  # the order is critical!
    ),
    {},
)

AttributeError: module 'openclsim.plugins' has no attribute 'HasWeatherPluginShiftAmountActivity'

In [ ]:
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "geometry": location_from_site,
    "capacity": 40,
    "level": 40,
}


data_to_site = {
    "env": my_env,
    "name": "Dumplocatie",
    "geometry": location_to_site,
    "capacity": 40,
    "level": 0,
}

from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

In [ ]:
data_hopper = {
    "env": my_env,
    "name": "Hopper 01",
    "geometry": location_from_site,
    "capacity": 4,
    "compute_v":lambda x: 10,
}

hopper = TransportProcessingResource(**data_hopper)

In [ ]:
metocean_df = pd.read_csv("../demo/unit_test_weather.csv")
metocean_df = metocean_df.set_index(pd.to_datetime(metocean_df["Time"], dayfirst=True))
metocean_df = metocean_df.sort_index()

metocean_df["Hs [m]"] = 4 + \
                        1.5*np.sin(metocean_df['  Hour']/24 * 8 *np.pi) + \
                        1.5*np.sin(metocean_df['  Hour']/24 * 6 *np.pi)

metocean_df["DateTime"] = [datetime.datetime.strptime(dt, '%d-%m-%Y %H:%M') for dt in metocean_df["Time"]]

In [ ]:
sailing_crit = core.WorkabilityCriterion(**{
    "event_name":"this",
    "condition":"Hs [m]",
    "maximum":6,
    "window_length":datetime.timedelta(hours=1)
})

loading_crit = core.WorkabilityCriterion(**{
    "event_name":"this",
    "condition":"Hs [m]",
    "maximum":4.5,
    "window_length":datetime.timedelta(hours=1)
})

In [ ]:
single_run = [
    TestMoveActivity(**{
        "env": my_env,
        "name": "Soil movement",
        "registry": registry,
        "mover": hopper,
        "destination": from_site,
        "metocean_criteria":[sailing_crit],
        "metocean_df": metocean_df,
        "timestep":1,
        "postpone_start": True,
    }),
    TestShiftActivity(**{
        "env": my_env,
        "name": "Transfer MP",
        "registry": registry,
        "processor": hopper,
        "origin": from_site,
        "destination": hopper,
        "amount": 4,
        "duration": 3600,
        "postpone_start": True,
        "metocean_criteria":[loading_crit],
        "metocean_df": metocean_df,
        "timestep":1,
    }),
    TestMoveActivity(**{
        "env": my_env,
        "name": "Soil movement",
        "registry": registry,
        "mover": hopper,
        "destination": to_site,
        "metocean_criteria":[sailing_crit],
        "metocean_df": metocean_df,
        "postpone_start": True,
        "timestep":1,
    }),
    TestShiftActivity(**{
        "env": my_env,
        "name": "Transfer TP",
        "registry": registry,
        "processor": hopper,
        "origin": hopper,
        "destination": to_site,
        "amount": 4,
        "duration": 3600,
        "postpone_start": True,
        "metocean_criteria":[loading_crit],
        "metocean_df": metocean_df,
        "timestep":1,
    })
]

activity = model.SequentialActivity(**{
    "env": my_env,
    "name": "Single run process",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff60",
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
})

expr = [{"type":"container", "concept": to_site, "state":"full"}]
while_data = {
    "env": my_env,
    "name": "while",
    "registry": registry,
    "sub_process": activity,
    "condition_event": expr,
    "postpone_start": False,
}
while_activity = model.WhileActivity(**while_data)


In [ ]:
my_env.run()

In [ ]:
objects = [hopper, from_site, to_site]

activities = []
for obj in objects:
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

timestamps = []
logs = [o.log["Timestamp"] for o in objects]
for log in logs:
    timestamps.extend(log)

fig = vessel_planning(objects, activities, static=True, y_scale="numbers")
fig.add_scatter(
    x=[min(timestamps), max(timestamps)],
    y=[sailing_crit.maximum,sailing_crit.maximum], 
    mode='lines',
    name= "sailing criterea"
)
fig.add_scatter(
    x=[min(timestamps), 
       max(timestamps)], 
    y=[loading_crit.maximum,loading_crit.maximum], 
    mode='lines', 
    name= "loading criterea"
)
fig.add_scatter(x=metocean_df["DateTime"], y=metocean_df["Hs [m]"], mode='lines', name = "Hs [m]")